In [1]:
### Import Packages

# File manipulation

import os # For working with Operating System
import requests # Accessing the Web
import datetime as dt # Working with dates/times

# Database 

import psycopg2
from psycopg2 import sql

# Analysis

import numpy as np
import arcpy
import pandas as pd
from scipy.spatial.distance import cdist
import pyproj


import os # For working with Operating System
import shutil # For deleting folders
import urllib # For accessing websites
import zipfile # For extracting from Zipfiles
from io import BytesIO # For reading bytes objects

# Database 

import psycopg2

# Analysis

import arcpy
import pandas as pd

In [2]:
# Get CWD

cwd = os.getcwd() # This is a global variable for where the notebook is (must change if running in arcpro)

# Create GeoDataBase
# This is the communal GeoDataBase

if not os.path.exists(os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')): # If it doesn't exist, create it

    arcpy.management.CreateFileGDB(os.path.join(cwd, '..', '..', 'data'), 'QAQC')

# Make it workspace



arcpy.env.overwriteOutput = True # Overwrite layers is okay

save_path_gdb = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

# Bring in station location data

In [9]:
df = pd.read_csv("stations_5-2.csv")
df

,sensor_index,last_modified,date_created,last_seen,name,firmware_version,uptime,position_rating,latitude,longitude,altitude,channel_state,channel_flags
0,3088,2017-09-09 21:42:29,2017-08-29 21:03:53,2023-05-02 19:42:13,Howe Neighborhood,6.06b,8871,5,44.935818,-93.217520,833,3,0
1,11134,2018-06-26 01:44:59,2018-05-22 21:13:09,2023-05-02 19:42:28,Linden Hills,7.02,57395,5,44.927760,-93.322350,886,3,1
2,142718,2023-02-02 17:31:01,2022-01-12 18:57:49,2023-05-02 19:41:57,City of Minneapolis Community Air Monitoring P...,7.02,61917,5,44.995792,-93.295395,865,3,0
3,142720,2023-02-02 17:31:45,2022-01-12 18:57:55,2023-05-02 19:42:47,City of Minneapolis community air monitoring p...,7.02,44937,5,44.956170,-93.254710,856,3,0
4,142726,2023-02-02 17:31:06,2022-01-12 18:58:17,2023-05-02 19:41:47,City of Minneapolis Community Air Monitoring P...,7.02,28347,5,45.015070,-93.289030,889,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,110979,2021-08-12 00:54:42,2021-06-26 00:00:15,2023-05-02 19:41:35,Washington St NE,7.02,11127,5,44.997475,-93.257260,847,3,0
76,113486,2021-08-06 18:31:12,2021-07-09 20:26:11,2023-04-28 01:53:16,Bancroft,7.02,36807,5,44.928123,-93.253890,821,3,0
77,116413,2022-10-29 18:06:01,2021-07-26 23:11:28,2023-05-02 19:43:08,Seward,7.02,62153,5,44.961216,-93.242400,839,3,0
78,126487,2022-06-28 01:26:03,2021-09-08 21:44:43,2023-05-02 19:41:45,Northwoods Berry Clan,7.02,33393,5,45.040394,-93.213270,939,3,0


# Analysis of PM 2.5 in Minneapolis 

### This notebook models the spread of PM 2.5 throughout Minneapolis using a Huff gravity model. An accuracy assessment is also completed on the model.

### Bring in data of pm 2.5 spikes

In [10]:
spikeData = pd.read_csv('Parameter_df.csv')
spikeData

,sensor_index,date,is_weekday,n_observations,pm25_fullDay_mean,pm25_fullDay_minutesAbove12ug,n_spikes,humidity_fullDay_mean,temperature_fullDay_mean,pressure_fullDay_mean,idwSum_facilities,idwSum_traffic
0,143656,2023-03-18,0,144,5.341125,190,0,45.833,23.505,988.205,0.005054,861.930809
1,142774,2022-11-05,0,144,8.901722,370,0,62.438,42.682,973.451,0.004609,452.986261
2,145454,2022-12-06,1,144,11.355750,690,2,48.174,26.181,987.556,67.350629,587.483355
3,142748,2022-12-07,1,144,11.848437,560,3,53.589,25.477,994.330,2.827060,3827.349398
4,143636,2023-01-22,0,144,24.739590,1440,50,60.596,28.776,983.963,24.669941,453.501189
...,...,...,...,...,...,...,...,...,...,...,...,...
10367,142728,2023-01-15,0,144,17.993132,1400,4,58.071,37.246,979.981,52.390284,2373.255885
10368,142774,2023-01-08,0,144,74.232264,1440,144,55.522,18.940,994.828,0.004609,452.986261
10369,143226,2022-06-26,0,144,3.610160,0,0,37.410,76.454,989.615,0.183773,342.481335
10370,143214,2023-03-09,1,144,8.922403,340,0,61.694,39.656,999.879,0.308809,327.047615


# Bring in wind speed averages for mlps

## Average them into daily wind speed and direction

In [11]:
# Load Data
msp_url = 'https://www.ncei.noaa.gov/data/normals-hourly/1991-2020/access/USW00014922.csv'
response = requests.get(msp_url)
msp_wind = pd.read_csv(msp_url, usecols=['STATION', 'LATITUDE', 'LONGITUDE', 'DATE', 'month', 'day', 'hour', 'HLY-WIND-AVGSPD', 'HLY-WIND-VCTDIR'])

# Rename columns
msp_wind = msp_wind.rename(columns={'month': 'MONTH', 'hour':'HOUR', 'day':'DAY', 'HLY-WIND-AVGSPD':'HLY_WIND_AVGSPD', 'HLY-WIND-VCTDIR':'HLY_WIND_VCDIR'})

# Convert 'DATE' to datetime format
msp_wind['DATE'] = pd.to_datetime(msp_wind['DATE'], format='%m-%dT%H:%M:%S')

# Group by station and date
grouped_wind_data = msp_wind.groupby([msp_wind.STATION, msp_wind.DATE.dt.date]).mean().reset_index()

# Rename the 'DATE' column to 'date' for consistency
grouped_wind_data = grouped_wind_data.rename(columns={'DATE': 'date'})

# Calculate daily averages
grouped_wind_data['daily_wind_speed_avg'] = grouped_wind_data['HLY_WIND_AVGSPD']
grouped_wind_data['daily_wind_vector_direction_avg'] = grouped_wind_data['HLY_WIND_VCDIR']

# Keep only relevant columns
daily_wind_data = grouped_wind_data[['STATION', 'date', 'daily_wind_speed_avg', 'daily_wind_vector_direction_avg']]

print(daily_wind_data)


         STATION        date  daily_wind_speed_avg  \
0    USW00014922  1900-01-01              9.075000   
1    USW00014922  1900-01-02              9.150000   
2    USW00014922  1900-01-03              9.216667   
3    USW00014922  1900-01-04              9.216667   
4    USW00014922  1900-01-05              9.275000   
..           ...         ...                   ...   
360  USW00014922  1900-12-27              9.170833   
361  USW00014922  1900-12-28              9.183333   
362  USW00014922  1900-12-29              9.154167   
363  USW00014922  1900-12-30              9.183333   
364  USW00014922  1900-12-31              9.108333   

     daily_wind_vector_direction_avg  
0                         287.666667  
1                         286.875000  
2                         284.958333  
3                         286.875000  
4                         288.125000  
..                               ...  
360                       286.625000  
361                       291.166667  


<ipython-input-11-d9f4aba99470>:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_wind_data = msp_wind.groupby([msp_wind.STATION, msp_wind.DATE.dt.date]).mean().reset_index()


### Join the wind and spike data

In [12]:
# Convert the 'date' columns in both DataFrames to datetime objects
spikeData['date'] = pd.to_datetime(spikeData['date'])
daily_wind_data['date'] = pd.to_datetime(daily_wind_data['date'])

# Extract the month and day from the 'date' columns
spikeData['month'] = spikeData['date'].dt.month
spikeData['day'] = spikeData['date'].dt.day

daily_wind_data['month'] = daily_wind_data['date'].dt.month
daily_wind_data['day'] = daily_wind_data['date'].dt.day

# Merge the DataFrames based on the 'month' and 'day' columns
merged_data = spikeData.merge(daily_wind_data, on=['month', 'day'], suffixes=('', '_wind'))

# Drop unnecessary columns
merged_data = merged_data.drop(columns=['STATION', 'date_wind'])

# Display the merged DataFrame
print(merged_data)


       sensor_index       date  is_weekday  n_observations  pm25_fullDay_mean  \
0            143656 2023-03-18           0             144           5.341125   
1            142730 2023-03-18           0             144           2.830403   
2            143214 2023-03-18           0             144           2.742826   
3            145242 2023-03-18           0             144           4.682944   
4            142774 2023-03-18           0             144           2.842639   
...             ...        ...         ...             ...                ...   
10367        142720 2022-06-22           1             144           3.474497   
10368        142724 2022-06-22           1             144           3.284281   
10369        143214 2022-06-22           1             144           2.914250   
10370        145242 2022-06-22           1             144           4.760962   
10371        145470 2022-06-22           1             144           4.151205   

       pm25_fullDay_minutes

<ipython-input-12-bd4ce85cf798>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_wind_data['date'] = pd.to_datetime(daily_wind_data['date'])
<ipython-input-12-bd4ce85cf798>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_wind_data['month'] = daily_wind_data['date'].dt.month
<ipython-input-12-bd4ce85cf798>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

### Join station location with joined spike and wind data

In [13]:
# Convert the 'sensor_index' columns to the same data type (e.g., int64)
df['sensor_index'] = df['sensor_index'].astype('int64')
merged_data['sensor_index'] = merged_data['sensor_index'].astype('int64')

# Merge the DataFrames based on the 'sensor_index' column
stationSpikeWindData = merged_data.merge(df, on='sensor_index')

# stationSpikeWindData.to_csv('station_spike_wind_data.csv', index=False)


print(stationSpikeWindData)

       sensor_index       date  is_weekday  n_observations  pm25_fullDay_mean  \
0            143656 2023-03-18           0             144           5.341125   
1            143656 2022-11-05           0             144           8.760681   
2            143656 2022-12-06           1             144           9.107910   
3            143656 2022-12-07           1             144          14.958146   
4            143656 2023-01-22           0             144          26.534174   
...             ...        ...         ...             ...                ...   
10328        145604 2023-04-17           1             144           2.503681   
10329        145604 2023-04-13           1             139           9.498234   
10330        145604 2023-04-15           0             142          10.386077   
10331        145604 2023-04-10           1             144          11.901583   
10332        145604 2023-04-20           1             144           8.795903   

       pm25_fullDay_minutes

## Normalize wind speed

In [14]:
# Assuming 'weight_speed' and 'weight_speed_direction' are column names in your merged_data DataFrame
weight_speed = stationSpikeWindData['daily_wind_speed_avg']

# Normalize weight_speed
weight_speed_min = weight_speed.min()
weight_speed_max = weight_speed.max()
stationSpikeWindData['weight_speed_normalized'] = (weight_speed - weight_speed_min) / (weight_speed_max - weight_speed_min)

# Print the merged_data DataFrame with the new columns 'weight_speed_normalized' and 'weight_speed_direction_normalized'
print(stationSpikeWindData)

       sensor_index       date  is_weekday  n_observations  pm25_fullDay_mean  \
0            143656 2023-03-18           0             144           5.341125   
1            143656 2022-11-05           0             144           8.760681   
2            143656 2022-12-06           1             144           9.107910   
3            143656 2022-12-07           1             144          14.958146   
4            143656 2023-01-22           0             144          26.534174   
...             ...        ...         ...             ...                ...   
10328        145604 2023-04-17           1             144           2.503681   
10329        145604 2023-04-13           1             139           9.498234   
10330        145604 2023-04-15           0             142          10.386077   
10331        145604 2023-04-10           1             144          11.901583   
10332        145604 2023-04-20           1             144           8.795903   

       pm25_fullDay_minutes

## Normalize wind direction

In [15]:
# Assuming 'wind_direction' is a column name in your stationSpikeWindData DataFrame
wind_direction = stationSpikeWindData['daily_wind_vector_direction_avg']

# Normalize wind_direction
wind_direction_normalized = wind_direction / 360

# Add the normalized wind direction as a new column to the stationSpikeWindData DataFrame
stationSpikeWindData['wind_direction_normalized'] = wind_direction_normalized

# Print the updated stationSpikeWindData DataFrame
print(stationSpikeWindData)

       sensor_index       date  is_weekday  n_observations  pm25_fullDay_mean  \
0            143656 2023-03-18           0             144           5.341125   
1            143656 2022-11-05           0             144           8.760681   
2            143656 2022-12-06           1             144           9.107910   
3            143656 2022-12-07           1             144          14.958146   
4            143656 2023-01-22           0             144          26.534174   
...             ...        ...         ...             ...                ...   
10328        145604 2023-04-17           1             144           2.503681   
10329        145604 2023-04-13           1             139           9.498234   
10330        145604 2023-04-15           0             142          10.386077   
10331        145604 2023-04-10           1             144          11.901583   
10332        145604 2023-04-20           1             144           8.795903   

       pm25_fullDay_minutes

# Historic Data daily average

In [16]:
## create new column of present absence based on N_spikes. 
stationSpikeWindData['present_absent'] = stationSpikeWindData['n_spikes'].apply(lambda x: 0 if x == 0 else 1)
stationSpikeWindData

,sensor_index,date,is_weekday,n_observations,pm25_fullDay_mean,pm25_fullDay_minutesAbove12ug,n_spikes,humidity_fullDay_mean,temperature_fullDay_mean,pressure_fullDay_mean,...,uptime,position_rating,latitude,longitude,altitude,channel_state,channel_flags,weight_speed_normalized,wind_direction_normalized,present_absent
0,143656,2023-03-18,0,144,5.341125,190,0,45.833,23.505,988.205,...,4468,5,44.932407,-93.28342,874,3,0,0.551807,0.548032,0
1,143656,2022-11-05,0,144,8.760681,290,0,60.125,44.010,971.738,...,4468,5,44.932407,-93.28342,874,3,0,0.577108,0.728472,0
2,143656,2022-12-06,1,144,9.107910,340,0,49.004,27.014,986.269,...,4468,5,44.932407,-93.28342,874,3,0,0.351807,0.753819,0
3,143656,2022-12-07,1,144,14.958146,610,14,49.725,28.465,993.579,...,4468,5,44.932407,-93.28342,874,3,0,0.374699,0.750579,1
4,143656,2023-01-22,0,144,26.534174,1430,55,61.631,28.517,984.403,...,4468,5,44.932407,-93.28342,874,3,0,0.500000,0.810995,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10328,145604,2023-04-17,1,144,2.503681,0,0,56.064,41.486,981.399,...,24060,5,44.961617,-93.23804,841,3,0,0.885542,0.116667,0
10329,145604,2023-04-13,1,139,9.498234,250,4,31.505,80.174,975.448,...,24060,5,44.961617,-93.23804,841,3,0,0.966265,0.092708,1
10330,145604,2023-04-15,0,142,10.386077,700,5,67.308,55.249,979.440,...,24060,5,44.961617,-93.23804,841,3,0,0.900000,0.091782,1
10331,145604,2023-04-10,1,144,11.901583,490,11,36.110,64.933,994.016,...,24060,5,44.961617,-93.23804,841,3,0,0.965060,0.118171,1


In [17]:
# create more columns of true postive, etc. these will be filled in later
stationSpikeWindData['overall_TP'] = 0
stationSpikeWindData['overall_FP'] = 0
stationSpikeWindData['overall_FN'] = 0
stationSpikeWindData['overall_TN'] = 0
stationSpikeWindData['Presence'] = 0
stationSpikeWindData['Accuracy'] = 0
stationSpikeWindData['Rank'] = 0
stationSpikeWindData

,sensor_index,date,is_weekday,n_observations,pm25_fullDay_mean,pm25_fullDay_minutesAbove12ug,n_spikes,humidity_fullDay_mean,temperature_fullDay_mean,pressure_fullDay_mean,...,weight_speed_normalized,wind_direction_normalized,present_absent,overall_TP,overall_FP,overall_FN,overall_TN,Presence,Accuracy,Rank
0,143656,2023-03-18,0,144,5.341125,190,0,45.833,23.505,988.205,...,0.551807,0.548032,0,0,0,0,0,0,0,0
1,143656,2022-11-05,0,144,8.760681,290,0,60.125,44.010,971.738,...,0.577108,0.728472,0,0,0,0,0,0,0,0
2,143656,2022-12-06,1,144,9.107910,340,0,49.004,27.014,986.269,...,0.351807,0.753819,0,0,0,0,0,0,0,0
3,143656,2022-12-07,1,144,14.958146,610,14,49.725,28.465,993.579,...,0.374699,0.750579,1,0,0,0,0,0,0,0
4,143656,2023-01-22,0,144,26.534174,1430,55,61.631,28.517,984.403,...,0.500000,0.810995,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10328,145604,2023-04-17,1,144,2.503681,0,0,56.064,41.486,981.399,...,0.885542,0.116667,0,0,0,0,0,0,0,0
10329,145604,2023-04-13,1,139,9.498234,250,4,31.505,80.174,975.448,...,0.966265,0.092708,1,0,0,0,0,0,0,0
10330,145604,2023-04-15,0,142,10.386077,700,5,67.308,55.249,979.440,...,0.900000,0.091782,1,0,0,0,0,0,0,0
10331,145604,2023-04-10,1,144,11.901583,490,11,36.110,64.933,994.016,...,0.965060,0.118171,1,0,0,0,0,0,0,0


In [18]:
# find population total for each sensor_index by summing N_spikes. this is our population

# divide by number of days they have been up. Have to find the nubmers of days each sensor has been up

total_n_spikes = stationSpikeWindData.groupby('sensor_index')['n_spikes'].agg(['sum', 'count']).reset_index()
total_n_spikes.columns = ['sensor_index', 'total_n_spikes','total_days']
total_n_spikes['spikes_per_day'] = total_n_spikes['total_n_spikes']/total_n_spikes['total_days']

total_n_spikes

,sensor_index,total_n_spikes,total_days,spikes_per_day
0,142718,4654,243,19.152263
1,142720,4363,316,13.806962
2,142724,335,178,1.882022
3,142726,4422,285,15.515789
4,142728,4379,222,19.725225
5,142730,2563,120,21.358333
6,142732,3668,292,12.561644
7,142734,3236,278,11.640288
8,142736,2438,133,18.330827
9,142744,3914,223,17.551570


In [19]:
# find distance between stations as a table

# Extract unique stations and their latitudes and longitudes
unique_stations = stationSpikeWindData[['sensor_index', 'latitude', 'longitude']].drop_duplicates()

# Convert latitude and longitude to Cartesian coordinates (x, y, z)
def latlon_to_xyz(lat, lon):
    proj = pyproj.Proj(proj='utm', zone=15, ellps='WGS84')
    x, y = proj(lon, lat)
    return x, y

unique_stations['x'], unique_stations['y'] = zip(*unique_stations.apply(lambda row: latlon_to_xyz(row['latitude'], row['longitude']), axis=1))

# Calculate pairwise distances between all stations
locations = unique_stations[['x', 'y']].values
distance_matrix = cdist(locations, locations, metric='euclidean')

# Create a DataFrame with the distance matrix
distance_df = pd.DataFrame(distance_matrix, columns=unique_stations['sensor_index'], index=unique_stations['sensor_index'])
distance_df

sensor_index,143656,142730,143214,145242,142774,143636,157935,145250,157871,145470,...,142726,145506,142728,145616,143942,143944,142724,166459,157757,145604
sensor_index,,,,,,,,,,,,,,,,,,,,,
143656,0.000000,6716.915796,4959.093743,8384.389769,3532.275768,8629.053685,3247.173800,4768.662836,1064.357853,4924.246726,...,9193.475201,2840.243830,4114.897578,5415.178397,5622.415595,4831.229429,3166.437375,4243.605066,3651.544166,4831.675609
142730,6716.915796,0.000000,10784.627490,2985.440459,4610.393571,1948.599124,7273.188115,3127.710367,6675.026944,10658.368203,...,2606.039440,9087.836354,5499.221049,1499.196709,1369.534977,6716.074165,7372.792659,10937.183558,5627.507384,5710.257459
143214,4959.093743,10784.627490,0.000000,11425.775457,8481.544812,12483.238182,7715.776460,9570.334628,4371.386510,211.402340,...,12877.667846,2155.298682,5839.230641,9794.030415,10009.310604,5295.466330,3547.415928,3066.951279,5483.835464,6156.776122
145242,8384.389769,2985.440459,11425.775457,0.000000,7230.682812,2404.988394,9760.564284,5956.427311,7978.953323,11259.438784,...,2156.856295,10158.408436,5596.483317,4248.563783,4202.712509,6440.145943,7881.580726,12324.555907,5943.481818,5416.900152
142774,3532.275768,4610.393571,8481.544812,7230.682812,0.000000,6499.450430,2680.682876,1638.576536,4210.605960,8436.430352,...,7199.426822,6372.474009,5647.093606,3114.496259,3245.049411,6847.821669,6047.335064,7470.791733,5378.125534,6299.272561
143636,8629.053685,1948.599124,12483.238182,2404.988394,6499.450430,0.000000,9176.945190,4939.218304,8515.952674,12343.223273,...,741.472337,10895.798878,6907.842847,3423.273866,3265.916036,8000.331381,8988.191042,12824.923448,7129.441588,6955.221864
157935,3247.173800,7273.188115,7715.776460,9760.564284,2680.682876,9176.945190,0.000000,4296.571152,4298.257538,7737.991379,...,9871.533569,5589.444226,6950.929545,5773.992599,5915.577558,7898.468383,6408.246357,5827.179317,6548.773716,7674.365865
145250,4768.662836,3127.710367,9570.334628,5956.427311,1638.576536,4939.218304,4296.571152,0.000000,5199.766684,9497.272136,...,5657.956436,7551.059738,5722.721048,1708.058505,1775.130736,7014.164232,6708.679508,8910.221375,5581.505184,6262.845435
157871,1064.357853,6675.026944,4371.386510,7978.953323,4210.605960,8515.952674,4298.257538,5199.766684,0.000000,4297.952035,...,9019.059731,2434.360721,3217.847972,5523.638153,5738.269143,3808.845324,2110.118973,4359.269893,2738.135681,3910.456355


In [81]:
# delete duplicate stations

# Melt the distance matrix DataFrame into a long format
distance_long = pd.melt(distance_df.reset_index(), id_vars=['sensor_index'], var_name='station_2', value_name='distance')
distance_long = distance_long.rename(columns={'sensor_index': 'station_1'})

# Remove duplicate station pairs
distance_long['station_pair'] = distance_long.apply(lambda row: frozenset([row['station_1'], row['station_2']]), axis=1)
distance_unique = distance_long.drop_duplicates(subset=['station_pair']).drop(columns=['station_pair'])

# Remove self-distances (i.e., distance of a station to itself)
distance_unique = distance_unique[distance_unique['station_1'] != distance_unique['station_2']]
distance_unique

,station_1,station_2,distance
1,142730,143656,6716.915796
2,143214,143656,4959.093743
3,145242,143656,8384.389769
4,142774,143656,3532.275768
5,143636,143656,8629.053685
...,...,...,...
2752,157757,142724,1938.099172
2753,145604,142724,2749.095572
2806,157757,166459,6681.443427
2807,145604,166459,7696.486014


# Bring in all spike data. And choose a random date to run analysis on

In [54]:
# All spikes read in csv

allSpikeDf = pd.read_csv('all_spikes.csv')
allSpikeDf['timestamp'] = pd.to_datetime(allSpikeDf.timestamp)
allSpikeDf

# Work to select a random day and finding earliest spike of 10 min interval
first_date = dt.datetime(2022, 6, 15) # June 15th, 2022?
datelist = pd.date_range(start = first_date, 
    end = dt.datetime.today(),
    normalize = True)
# print('Last Run on ', dt.datetime.today())

# select random date
rand_index = int(np.random.uniform(0, len(datelist),1)[0])
datelist[rand_index]

# select data from this random day
previous_readings = allSpikeDf[(allSpikeDf.timestamp.dt.date == datelist[rand_index].date())]
init_time = previous_readings.timestamp.min()

print('Selected day' , init_time)

print('Print list of selected spikes:', len(previous_readings))

Selected day 2023-02-02 00:00:00
Print list of selected spikes: 3296


### sum all spikes within that two hour time span by sensor index

In [82]:
# Convert 'timestamp' to datetime
allSpikeDf['timestamp'] = pd.to_datetime(allSpikeDf['timestamp'])

# Define the start and end times for the desired period
start_time = pd.Timestamp('2023-02-02 00:00:00')
end_time = pd.Timestamp('2023-02-02 02:00:00')

# Filter the DataFrame to include only the desired timestamps
filtered_df = allSpikeDf[(allSpikeDf['timestamp'] >= start_time) & (allSpikeDf['timestamp'] <= end_time)]

# Compute the count of 'pm25' values for each 'sensor_index' in the filtered DataFrame
originalPm25 = filtered_df.groupby('sensor_index').size().reset_index(name='originalPm2.5')

# Assuming 'stationSpikeWindData' is the other dataframe you mentioned
# Merging dataframes
originalPm2_5Spikes = pd.merge(originalPm25, df[['sensor_index', 'latitude', 'longitude']].drop_duplicates(), on='sensor_index', how='left')
originalPm2_5Spikes = originalPm2_5Spikes.dropna(subset=['latitude', 'longitude'])
originalPm2_5Spikes

,sensor_index,originalPm2.5,latitude,longitude
0,142718,5,44.995792,-93.295395
1,142720,4,44.956170,-93.254710
2,142726,5,45.015070,-93.289030
3,142728,4,44.958040,-93.245766
4,142730,4,44.992180,-93.296270
5,142732,4,44.916794,-93.273834
6,142734,3,44.903934,-93.280910
7,142736,6,44.972507,-93.283010
8,142744,4,44.996160,-93.296550
9,142748,2,44.965630,-93.284880


In [83]:
# create inital time of spike

init_presences = previous_readings[previous_readings.timestamp == init_time].sensor_index
init_presences

129116    145454
Name: sensor_index, dtype: int64

In [84]:
# add probabilty column to station distance table

distance_unique['MovementProbability'] = 0
distance_unique['Presence'] = 0
distance_unique

,station_1,station_2,distance,MovementProbability,Presence
1,142730,143656,6716.915796,0,0
2,143214,143656,4959.093743,0,0
3,145242,143656,8384.389769,0,0
4,142774,143656,3532.275768,0,0
5,143636,143656,8629.053685,0,0
...,...,...,...,...,...
2752,157757,142724,1938.099172,0,0
2753,145604,142724,2749.095572,0,0
2806,157757,166459,6681.443427,0,0
2807,145604,166459,7696.486014,0,0


In [85]:
# Merge distance_unique with total_n_spikes on station_1 and sensor_index
distance_unique = distance_unique.merge(total_n_spikes, left_on='station_1', right_on='sensor_index', how='left').copy()
distance_unique = distance_unique.rename(columns={'spikes_per_day': 'station_1_spikes_per_day'})

# Drop unnecessary columns
distance_unique = distance_unique.drop(columns=['sensor_index', 'total_n_spikes', 'total_days'])

# Merge distance_unique with total_n_spikes on station_2 and sensor_index
distance_unique = distance_unique.merge(total_n_spikes, left_on='station_2', right_on='sensor_index', how='left').copy()
distance_unique = distance_unique.rename(columns={'spikes_per_day': 'station_2_spikes_per_day'})

# Drop unnecessary columns
distance_unique = distance_unique.drop(columns=['sensor_index', 'total_n_spikes', 'total_days'])

# Display the resulting dataframe
distance_unique

,station_1,station_2,distance,MovementProbability,Presence,station_1_spikes_per_day,station_2_spikes_per_day
0,142730,143656,6716.915796,0,0,21.358333,20.983871
1,143214,143656,4959.093743,0,0,11.858491,20.983871
2,145242,143656,8384.389769,0,0,17.825532,20.983871
3,142774,143656,3532.275768,0,0,20.737430,20.983871
4,143636,143656,8629.053685,0,0,13.704861,20.983871
...,...,...,...,...,...,...,...
1426,157757,142724,1938.099172,0,0,2.074074,1.882022
1427,145604,142724,2749.095572,0,0,2.653846,1.882022
1428,157757,166459,6681.443427,0,0,2.074074,2.638889
1429,145604,166459,7696.486014,0,0,2.653846,2.638889


# Run Huff Model

## This helped us frame and understand how to run the model
#### https://gisgeography.com/huff-gravity-model/

In [86]:
sensor_indexes = distance_unique.station_1.unique()

In [151]:
def run_single_simulation(df, alpha, beta):
    distance_unique_simulation = df.copy()
    distance_unique_simulation['attractiveness'] = distance_unique_simulation.station_2_spikes_per_day.astype(int)**alpha/distance_unique_simulation.distance.astype(int)**beta
    distance_unique_simulation['MovementProbability'] = distance_unique_simulation.attractiveness / distance_unique_simulation.attractiveness.sum()
    distance_unique_simulation['Presence'] = np.random.rand(len(distance_unique_simulation)) < distance_unique_simulation['MovementProbability']
    distance_unique_simulation['Presence'] = distance_unique_simulation['Presence'].astype(int)
    return distance_unique_simulation

n_simulations = 100
n_iterations = 12

simulation_results = []

for sim in range(n_simulations):
    simulation_i_result = [distance_unique.copy()]
    for i in range(n_iterations):
        # print(simulation_i_result[i])
        simulation = run_single_simulation(simulation_i_result[i], 1, 2)
        simulation_i_result.append(simulation)
    simulation_results.append(simulation_i_result)

# Combine all simulation results
# combined_results = pd.concat(simulation_results, ignore_index=True)
# combined_results


In [152]:
# how many present and absent
combined_results.Presence.value_counts()

0    142999
1       101
Name: Presence, dtype: int64

In [153]:
# end time after 120 minutes

original_predicted_results= []

for simulation_result in simulation_results:
    
    end_time = init_time + dt.timedelta(minutes = (n_iterations) * 10)

    # select out spikes
    times = pd.date_range(start = init_time, 
                    end = end_time, freq = '10min')


    # Iterate through time stamps and identify the true presence of PM2.5 at each station

    sim_results = []

    for i, time in enumerate(times[1:]):

        true_presences = previous_readings[previous_readings.timestamp == time].sensor_index

        temp = simulation_result[i + 1].copy() 

        temp['True Presence'] = 0

        condition = temp.station_1.isin(true_presences)
        temp.loc[condition, 'True Presence'] = 1

        sim_results.append(temp)


    # combine orignal and predicted

    original_predicted_result = pd.concat(sim_results, ignore_index=True)

    y_true = original_predicted_result['True Presence'] 
    y_pred = original_predicted_result['Presence']
    
    original_predicted_results.append(original_predicted_result)

# Accuraccy assesment

In [154]:
# accuracy assesment for alpha 2

for original_predicted_result in original_predicted_results:
    
    # Update the table with the computed values
    for station_index in sensor_indexes:
        select_df = original_predicted_result[original_predicted_result.station_1==station_index]
        y_true = select_df['True Presence'] 
        y_pred = select_df['Presence']

        TP = np.sum((y_true == 1) & (y_pred == 1))
        FP = np.sum((y_true == 0) & (y_pred == 1))
        FN = np.sum((y_true == 1) & (y_pred == 0))
        TN = np.sum((y_true == 0) & (y_pred == 0))


        original_predicted_result.loc[original_predicted_result['station_1'] == station_index, 'TP'] = TP
        original_predicted_result.loc[original_predicted_result['station_1'] == station_index, 'FP'] = FP
        original_predicted_result.loc[original_predicted_result['station_1'] == station_index, 'FN'] = FN
        original_predicted_result.loc[original_predicted_result['station_1'] == station_index, 'TN'] = TN

    # Calculate accuracy
    original_predicted_result['Accuracy'] = (original_predicted_result['TP'] + original_predicted_result['TN']) / (original_predicted_result['TP'] + original_predicted_result['FP'] + original_predicted_result['FN'] + original_predicted_result['TN'])
    original_predicted_result

In [155]:
original_predicted_result

,station_1,station_2,distance,MovementProbability,Presence,station_1_spikes_per_day,station_2_spikes_per_day,attractiveness,True Presence,TP,FP,FN,TN,Accuracy
0,142730,143656,6716.915796,0.000077,0,21.358333,20.983871,4.434132e-07,0,0.0,0.0,4.0,8.0,0.666667
1,143214,143656,4959.093743,0.000141,0,11.858491,20.983871,8.132832e-07,0,0.0,0.0,8.0,16.0,0.666667
2,145242,143656,8384.389769,0.000049,0,17.825532,20.983871,2.845296e-07,0,0.0,0.0,18.0,18.0,0.500000
3,142774,143656,3532.275768,0.000277,0,20.737430,20.983871,1.603203e-06,0,0.0,0.0,24.0,24.0,0.500000
4,143636,143656,8629.053685,0.000046,0,13.704861,20.983871,2.686019e-07,0,0.0,0.0,20.0,40.0,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17167,157757,142724,1938.099172,0.000046,0,2.074074,1.882022,2.662517e-07,0,0.0,1.0,0.0,623.0,0.998397
17168,145604,142724,2749.095572,0.000023,0,2.653846,1.882022,1.323276e-07,0,0.0,0.0,0.0,636.0,1.000000
17169,157757,166459,6681.443427,0.000008,0,2.074074,2.638889,4.480712e-08,0,0.0,1.0,0.0,623.0,0.998397
17170,145604,166459,7696.486014,0.000006,0,2.653846,2.638889,3.376758e-08,0,0.0,0.0,0.0,636.0,1.000000


In [156]:
# for loop of accuracy assessment
precisionAlpha2s = []
recallAlpha2s = []
alpha2AverageAccuracys = []

for original_predicted_result in original_predicted_results:
    
    precisionAlpha2 = original_predicted_result['TP'].sum() / (original_predicted_result['TP'].sum() + original_predicted_result['FP'].sum())
    precisionAlpha2s.append(precisionAlpha2)

    recallAlpha2 = original_predicted_result['TP'].sum() / (original_predicted_result['TP'].sum() + original_predicted_result['FN'].sum())
    recallAlpha2s.append(recallAlpha2)
    
    original_predicted_result['TP'].sum()

    alpha2AverageAccuracy = original_predicted_result['Accuracy'].mean()
    alpha2AverageAccuracys.append(alpha2AverageAccuracy)

In [157]:
# results of accuracy assessment
mean_precisionAlpha2s = numpy.mean(precisionAlpha2s)
print(mean_precisionAlpha2s)
mean_recallAlpha2s = numpy.mean(recallAlpha2s)
print(mean_recallAlpha2s)
mean_alpha2AverageAccuracys =  numpy.mean(alpha2AverageAccuracys)
print(mean_alpha2AverageAccuracys)

0.26127653971694664
0.0006987896177566704
0.7306184486373166


In [158]:
best_index = numpy.argmax(precisionAlpha2s)
best_simulation = original_predicted_results[best_index]
best_simulation

,station_1,station_2,distance,MovementProbability,Presence,station_1_spikes_per_day,station_2_spikes_per_day,attractiveness,True Presence,TP,FP,FN,TN,Accuracy
0,142730,143656,6716.915796,0.000077,0,21.358333,20.983871,4.434132e-07,0,0.0,0.0,4.0,8.0,0.666667
1,143214,143656,4959.093743,0.000141,0,11.858491,20.983871,8.132832e-07,0,0.0,0.0,8.0,16.0,0.666667
2,145242,143656,8384.389769,0.000049,0,17.825532,20.983871,2.845296e-07,0,0.0,0.0,18.0,18.0,0.500000
3,142774,143656,3532.275768,0.000277,0,20.737430,20.983871,1.603203e-06,0,0.0,0.0,24.0,24.0,0.500000
4,143636,143656,8629.053685,0.000046,0,13.704861,20.983871,2.686019e-07,0,0.0,0.0,20.0,40.0,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17167,157757,142724,1938.099172,0.000046,0,2.074074,1.882022,2.662517e-07,0,0.0,0.0,0.0,624.0,1.000000
17168,145604,142724,2749.095572,0.000023,0,2.653846,1.882022,1.323276e-07,0,0.0,0.0,0.0,636.0,1.000000
17169,157757,166459,6681.443427,0.000008,0,2.074074,2.638889,4.480712e-08,0,0.0,0.0,0.0,624.0,1.000000
17170,145604,166459,7696.486014,0.000006,0,2.653846,2.638889,3.376758e-08,0,0.0,0.0,0.0,636.0,1.000000


In [159]:
results_groupeds = []

for original_predicted_result in original_predicted_results:
    results_grouped = original_predicted_result.groupby(['station_1', 'station_2']).sum()['Presence'].reset_index()
    results_groupeds.append(results_grouped)

## find predicted PM2.5 spikes

In [160]:
predictedPM2_5 = df[['sensor_index', 'latitude', 'longitude']].copy()
predictedPM2_5['mean_Presence'] = None


for sensor_index in df.sensor_index.unique():
    sensor_Presences = []
    for results_grouped in results_groupeds:
        if sensor_index in results_grouped.station_1.tolist():
            sensor_Presence = results_grouped[results_grouped.station_1 == sensor_index]['Presence'].iloc[0]
            sensor_Presences.append(sensor_Presence)
    sensor_Presence_mean = numpy.mean(sensor_Presences)
    predictedPM2_5.loc[predictedPM2_5.sensor_index == sensor_index, 'mean_Presence'] = sensor_Presence_mean

# # Merge 'PredictedsumPresence' with 'df' on 'station_1'/'sensor_index'
# predictedPM2_5 = pd.merge(results_grouped, df[['sensor_index', 'latitude', 'longitude']], 
#                       left_on='station_1', right_on='sensor_index', how='left').drop(columns='sensor_index')

# # Drop duplicates based on 'station_1'
# predictedPM2_5 = predictedPM2_5.drop_duplicates(subset='station_1')
# predictedPM2_5

C:\Users\swimb\AppData\Local\ESRI\conda\envs\arcpy_clone_ethan\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\swimb\AppData\Local\ESRI\conda\envs\arcpy_clone_ethan\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [183]:
predictedPM2_5 = predictedPM2_5.dropna()
predictedPM2_5

,sensor_index,latitude,longitude,mean_Presence,WKT
2,142718,44.995792,-93.295395,0.2,POINT (-93.295394999999999 44.995792000000002)
3,142720,44.956170,-93.254710,0.01,POINT (-93.254710000000003 44.95617)
4,142726,45.015070,-93.289030,0.0,POINT (-93.289029999999997 45.015070000000001)
5,142724,44.937218,-93.243866,0.0,POINT (-93.243865999999997 44.937218000000001)
6,142730,44.992180,-93.296270,0.0,POINT (-93.296270000000007 44.992179999999998)
7,142728,44.958040,-93.245766,0.01,POINT (-93.245766000000003 44.958039999999997)
8,142734,44.903934,-93.280910,0.0,POINT (-93.280910000000006 44.903934)
9,142732,44.916794,-93.273834,0.0,POINT (-93.273833999999994 44.916794000000003)
10,142736,44.972507,-93.283010,0.01,POINT (-93.283010000000004 44.972507)
11,142744,44.996160,-93.296550,3.96,POINT (-93.296549999999996 44.996160000000003)


In [162]:
# Create a list to store the WKT values for predicted spikes
wkt_list = []
# Iterate over each row and add the WKT representation of a point geometry
for index, row in predictedPM2_5.iterrows():
    latitude, longitude = row['latitude'], row['longitude']
    point = arcpy.Point(longitude, latitude)
    point_geom = arcpy.PointGeometry(point, arcpy.SpatialReference(4326))    
    wkt = point_geom.WKT    
    wkt_list.append(wkt)
    
# Add the WKT column to the DataFrame
predictedPM2_5['WKT'] = wkt_list

In [174]:
# Create a list to store the WKT values for orignal actual spikes pm2.5
wkt_list = []
# Iterate over each row and add the WKT representation of a point geometry
for index, row in originalPm2_5Spikes.iterrows():
    latitude, longitude = row['latitude'], row['longitude']
    point = arcpy.Point(longitude, latitude)
    point_geom = arcpy.PointGeometry(point, arcpy.SpatialReference(4326))    
    wkt = point_geom.WKT    
    wkt_list.append(wkt)
    
# Add the WKT column to the DataFrame
originalPm2_5Spikes['WKT'] = wkt_list

# Save to Local and Remote Databases

In [184]:
# Get credentials

cred_pth = os.path.join(os.getcwd(), '..', '..', 'database', 'db_credentials.txt')

with open(cred_pth, 'r') as f:
    
    creds = f.readlines()[0].split(', ')

# Connect to PostGIS Database

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

try:
    conn = psycopg2.connect(**pg_connection_dict)
    print("connected")
except:
    print("connection failed")

connected


In [185]:
# Create and fill table for mean_Presence

points = os.path.join(save_path_gdb, "mean_Presence")

# For 'mean_Presence'
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS mean_Presence")
cursor.execute("""
    CREATE TABLE mean_Presence (
        id SERIAL,
        station_1 INT,
        mean_Presence FLOAT,
        geom geometry
        )
""")
conn.commit()

# Populate the table
for _, row in predictedPM2_5.iterrows():
    cursor.execute(
        "INSERT INTO mean_Presence (station_1, mean_Presence, geom) VALUES (%s, %s, ST_SetSRID(ST_GeomFromText(%s), 4326)::geometry)", 
        (row['sensor_index'], row['mean_Presence'], row['WKT'])
    )
conn.commit()

In [180]:
# Create and fill table for mean_Presence for original

points = os.path.join(save_path_gdb, "originalPm2_5Spikes")

# For 'original mean_Presence'
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS originalPm2_5")
cursor.execute("""
    CREATE TABLE originalPm2_5 (
        id SERIAL,
        sensor_index INT,
        originalPm2_5 FLOAT,
        geom geometry
        )
""")
conn.commit()

# Populate the table
for _, row in originalPm2_5Spikes.iterrows():
    cursor.execute(
        "INSERT INTO originalPm2_5 (sensor_index, originalPm2_5, geom) VALUES (%s, %s, ST_SetSRID(ST_GeomFromText(%s), 4326)::geometry)", 
        (row['sensor_index'], row['originalPm2.5'], row['WKT'])
    )
conn.commit()

In [186]:
conn.close()